In [1]:
# Importações essenciais para o projeto
import numpy as np
import pandas as pd
import os

# TensorFlow e Keras para construção e treinamento de modelos de rede neural
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16

# Scikit-learn para pré-processamento de dados, divisão de conjunto de dados e métricas de avaliação
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.utils import shuffle

# Bibliotecas de Otimização por Enxame de Partículas
from SwarmPackagePy import gwo
import pyswarms as ps

In [2]:
# Preparação dos Dados
def create_street_data(path, street_types, im_size):
    images, labels = [], []
    streets = [(item, os.path.join(path, item, street)) 
               for item in street_types 
               for street in os.listdir(os.path.join(path, item))]
    streets_df = pd.DataFrame(streets, columns=['street type', 'image'])
    
    for _, row in streets_df.iterrows():
        img = load_img(row['image'], target_size=(im_size, im_size))
        images.append(img_to_array(img))
        labels.append(row['street type'])
    
    return np.array(images, dtype='float32') / 255.0, np.array(labels)

In [3]:
# Define o tamanho da imagem
im_size = 224

# Define o tipo de imagem
"""street_types = ['Apple', 'Banana', 'Cocos']
path = '../DatasetFruits/'
path_test = '../DatasetFruits_Test/'"""

street_types = ['clean', 'litter', 'recycle']
path = '../Dataset/'
path_test = '../Dataset_Test/'

In [4]:
# Criar dados de treino, validação e teste
train_images, train_labels = create_street_data(path, street_types, im_size)
test_images, test_labels = create_street_data(path_test, street_types, im_size)

# Contar as ocorrencias de cada tipo de rua e mostrar
streets_count = pd.value_counts(train_labels)
print("Streets in each category:", streets_count)

Streets in each category: recycle    1500
clean      1460
litter     1364
dtype: int64


In [5]:
# Converter etiquetas para valores numéricos
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Baralhar e dividir os dados de treino e validação
train_x, val_x, train_y, val_y = train_test_split(train_images, train_labels_encoded, test_size=0.15, random_state=415)

# Imprimir as formas dos conjuntos de dados
print(f"Train shape: {train_x.shape}")
print(f"Validation shape: {val_x.shape}")
print(f"Test shape: {test_images.shape}")

Train shape: (3675, 224, 224, 3)
Validation shape: (649, 224, 224, 3)
Test shape: (1081, 224, 224, 3)


In [1]:
def train_network(hyperparameters, train_x, train_y, val_x, val_y, street_types):
    learning_rate, dropout_rate = hyperparameters  
    im_size = train_x.shape[1]
    
    # Carregar o modelo VGG16 pré-treinado, sem as camadas superiores
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

    # Congelar as camadas do modelo base
    for layer in base_model.layers:
        layer.trainable = False

    # Construção do modelo com as novas camadas
    x = base_model.output
    x = Flatten()(x)
    x = Dropout(dropout_rate)(x)  # Usar a variável desempacotada
    x = Dense(len(street_types), activation='softmax')(x)  # Saída com base no número de tipos de rua

    # Montar o modelo final
    model = Model(inputs=base_model.input, outputs=x)

    # Compilar o modelo
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    # Treinar o modelo
    model.fit(train_x, 
              train_y, 
              epochs=5, 
              validation_data=(val_x, val_y), 
              verbose=0)
    
    # Avaliar o modelo
    validation_loss, validation_accuracy = model.evaluate(val_x, val_y, verbose=0)
    return validation_loss

In [2]:
def fitness_function(hyperparameters):
    loss = train_network(hyperparameters, train_x, train_y, val_x, val_y, street_types)
    print(f"Particle: {hyperparameters}, Loss: {loss}")
    return loss

In [ ]:
# Parâmetros do GWO
n_agents = 5
n_iterations = 5
dimensions = 2
lower_bound = [0.00001, 0.0]  # Limites inferiores para taxa de aprendizado e dropout
upper_bound = [0.1, 0.5]     # Limites superiores para taxa de aprendizado e dropout

# Inicializar e executar o otimizador GWO
optimizer = gwo(n_agents, fitness_function, lower_bound, upper_bound, dimensions, n_iterations)

# Desempacotar os melhores híper-parâmetros encontrados
best_learning_rate, best_dropout_rate = optimizer.get_Gbest()

In [6]:
# Carregar o modelo VGG16 pré-treinado, sem as camadas superiores
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(im_size, im_size, 3))

# Congelar as camadas do modelo base
for layer in base_model.layers:
    layer.trainable = False

# Construção do modelo final com as melhores taxas de aprendizado e dropout encontradas pelo GWO
x = base_model.output
x = Flatten()(x)
x = Dropout(best_dropout_rate)(x)  # Usar o melhor dropout_rate encontrado pelo GWO
x = Dense(len(street_types), activation='softmax')(x)  # Saída com base no número de tipos de rua

# Montar o modelo final
final_model = Model(inputs=base_model.input, outputs=x)

# Compilar o modelo final com o melhor learning_rate encontrado pelo GWO
final_model.compile(optimizer=Adam(best_learning_rate),
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])

# Treinar o modelo final
history = final_model.fit(train_x, 
                          train_y, 
                          epochs=5,
                          validation_data=(val_x, val_y))

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
print(f"Melhor taxa de aprendizado: {best_learning_rate}")
print(f"Melhor taxa de dropout: {best_dropout_rate}")

# Avaliar o modelo final nos dados de teste
test_loss, test_accuracy = final_model.evaluate(test_images, test_labels_encoded, verbose=1)
print(f"\nAcurácia no teste: {test_accuracy*100:.2f}%\n")

# Realizar previsões nos dados de teste
y_pred_test = final_model.predict(test_images)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)

# Matriz de confusão
confusion_mtx = confusion_matrix(test_labels_encoded, y_pred_test_classes)
print("Matriz de Confusão:")
print(confusion_mtx)

# Relatório de classificação
class_report = classification_report(test_labels_encoded, y_pred_test_classes, target_names=label_encoder.classes_)
print("\nRelatório de Classificação:")
print(class_report)

# Binarizar as etiquetas para cálculo do AUC
y_true_binarized = to_categorical(test_labels_encoded)

# Calculando o AUC para cada classe
for i in range(y_true_binarized.shape[1]):  # Percorrer cada classe
    auc_score = roc_auc_score(y_true_binarized[:, i], y_pred_test[:, i])
    print(f'AUC para a classe {label_encoder.classes_[i]}: {auc_score:.2f}')

In [ ]:
# Definir o caminho e o nome do arquivo para salvar o modelo
model_save_path = 'IC_Project_Fase3_GWO_SaveModel.h5'

# Salvar o modelo
model.save(model_save_path)
print(f"Modelo salvo com sucesso em: {model_save_path}")